# Mundo 6 - Como pegar histórico de apostas com o selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os

In [ ]:
%load_ext dotenv

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
url = '''https://www.bet365.com/?#/ME/X8020'''

driver.get(url)

driver.maximize_window()

In [ ]:
usuario = os.getenv('usuario')
senha = os.getenv('senha')

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.NAME, "undefined")))

# Como preencher formulários?

Existem duas formas, igual o iframe.

# Forma 1:

In [ ]:
caixa_usuario = driver.find_element("xpath", '/html/body/div[1]/div/div[1]/div[2]/div/div/div/div/div[1]/div[2]/div/input')

caixa_usuario.send_keys(usuario)

In [ ]:
caixa_usuario.clear()

# Forma 2:

In [ ]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '//*[@id="password"]'))).send_keys(senha)

In [ ]:
#clicando no botão de login de uma forma diferente

WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div/div/div/div/div[2]/button'))).click() 

In [ ]:
#mudando iframe
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "undefined")))

    
#selecionando intervalo de datas  
WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[1]/div[3]'))).click()

#abrindo calendário
WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div[1]/div/div[2]'))).click()


In [ ]:
#voltando 5 meses
for i in range(5):
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]'))).click()


#selecionando o dia do mês
WebDriverWait(driver, 25).until(EC.element_to_be_clickable(
     (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/table/tbody/tr[1]/td[7]'))).click()

#clicando em "mostrar histórico"
WebDriverWait(driver, 25).until(EC.element_to_be_clickable(
     (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[3]'))).click()

In [ ]:
#abrindo histórico de apostas até onde da

while True:
    try:
        WebDriverWait(driver, 25).until(EC.element_to_be_clickable(
            (By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div/div[3]'))).click()
    except:
        break

In [ ]:
inspect = '/html/body/div[1]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div/div[2]'
element = driver.find_element('xpath', f'{inspect}')
html_element = element.get_attribute('outerHTML')
soup = BeautifulSoup(html_element, 'html.parser')

soup

In [ ]:
#pra isso o BS4 é útil. Você acha uma lista de coisas baseado nas classe. 

dados = {'datas': [], 'apostas': [], 'odds': [],
         'valores': [], 'retornos': [],  'detalhes': []}

datas = soup.find_all("div", {"class": "h-BetSummary_DateAndTime"})
valores = soup.find_all(
    "div", {"class": "h-StakeReturnSection_StakeContainer"})
retornos = soup.find_all("div", {"class": "h-StakeReturnSection_ReturnText"})
apostas = soup.find_all("div", {"class": "h-BetSummary"})
detalhes = soup.find_all("div", {"h-StakeDescription_Text"})

In [ ]:
valores[0].text

In [ ]:
#tentando fazer a mesma coisa no selenium:

datas_selenium = driver.find_elements(By.NAME, "h-StakeReturnSection_StakeContainer")

datas_selenium

#só pode ser via ID, nome ou xpath. Pra encontrar multiplos elementos é melhor usar o BS4 que ainda ter permite
#ter um duplo controle: eu quero uma div com a class com nome x. 

In [ ]:
i = 0

for data, valor, retorno, aposta, detalhe in zip(datas, valores, retornos, apostas, detalhes):

    tipos = aposta.find_all("div", {"class": "h-BetSelection_Name"})
    odds = aposta.find_all("div", {"class": "h-BetSelection_Odds"})
    
    #isso aqui pra conferir se é uma múltipla 
    
    if tipos == []:
        tipos = aposta.find_all(
            "div", {"class": "h-BetBuilderSelection_SelectionLabel"})
        odds = aposta.find_all(
            "div", {"class": "h-BetBuilderMultipleSelections_OddsLabel"})

    aposta_string = ""
    for possibilidades in tipos:
        aposta_string = aposta_string + " & " + possibilidades.text
    aposta_string = aposta_string[2:] #pra não ter que fazer um exceção de primeiro elemento no loop

    valores_odds = []
    
    for odd in odds:
        spans = odd.find_all('span')
        for span in spans:
            
            if i == 8:
                print(span)
                #isso aqui serve pra não pegar coisa errada quando existe um boost na aposta
            if not 'h-BetOdds' in str(span):
                valores_odds.append(float(span.text.replace(',', '.'))) #nós usamos "," para decimal
            valor_final = np.prod(np.array(valores_odds)) #pra calcular a multipla temos multiplicar as odds

    dados['apostas'].append(aposta_string)
    dados['datas'].append(data.text)
    dados['valores'].append(valor.text)
    dados['retornos'].append(retorno.text)
    dados['odds'].append(valor_final)
    dados['detalhes'].append(detalhe.text)
    
    i = i + 1

df = pd.DataFrame(dados)
display(df)